<a href="https://colab.research.google.com/github/adampotton/MDM3-Rep-3/blob/main/training/Ensemble_Model/ensemble_training_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files, drive
drive.mount('/content/drive')
import torch
import torch.nn as nn
import numpy as np
import json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading trained Aerial and sentinel models

In [2]:
aerial_model = torch.load("/content/drive/My Drive/Aerial Data/training_run_3_filtered_data.pth")
sentinel_model = torch.load("/content/drive/My Drive/S2 Data All/training_run_1_filtered_data_19.pth")

# Loading training and validation data
Data has been pre sorted so that there is correspondence between the aerial and Sentinel data.

In [3]:
with open('/content/drive/My Drive/Aerial Data/Both Imagery/datapoints.json', 'r') as f:
    data = json.load(f)

S2_arrays = np.load('/content/drive/My Drive/Aerial Data/Both Imagery/arrays.npy')
S2_arrays_cut = S2_arrays[[item[5] for item in data]]

# A_arrays = np.load('/content/drive/My Drive/Aerial Data/Both Imagery/aerial_99_images.npy')
# A_arrays_cut = A_arrays[[item[2] for item in data]]

# Defining Ensemble Model Structure

In [ ]:

class EnsembleModel(nn.Module):
    #output size is the number of category
    def __init__(self, sentinel_model, aerial_model, output_size):
        super().__init__()
        self.sentinel_model = sentinel_model
        self.aerial_model = aerial_model
        #Linear ensemble layer which we want to train
        self.classifier = nn.Linear(output_size*2, output_size)

    def forward(self, sentinel_image, aerial_image):
        pred1 = self.sentinel_model(sentinel_image)
        pred2 = self.aerial_model(aerial_image)
        #verify that dimensions are correct here
        joined_preds = torch.cat((pred1,pred2), dim=1)
        out = self.classifier(joined_preds)
        return out

ensemble_model = EnsembleModel(sentinel_model,aerial_model,output_size=19)

for param in ensemble_model.parameters():
    param.requires_grad = False

for param in ensemble_model.classifier.parameters():
    param.requires_grad = True

device
ensemble_model = ensemble_model.to(device)

# Training the ensemble model

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=True, tensorboard_writer=None, early_stopping=False, patience=5, stop_tol=1e-3):
    early_stop = False

    if tensorboard_writer is not None:
        use_tensorboard = True
        writer = tensorboard_writer
    else:
        use_tensorboard = False

    since = time.time()

    val_acc_history = []
    val_loss_history = []
    train_acc_history = []
    train_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    final_epoch = num_epochs

    for epoch in range(num_epochs):
        if early_stop:
            break
        else:
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        # Get model outputs and calculate loss
                        # Special case for inception because in training it has an auxiliary output. In train
                        #   mode we calculate the loss by summing the final output and the auxiliary output
                        #   but in testing we only consider the final output.
                        if is_inception and phase == 'train':
                            # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                            outputs, aux_outputs = model(inputs)
                            loss1 = criterion(outputs, labels)
                            loss2 = criterion(aux_outputs, labels)
                            loss = loss1 + 0.4*loss2
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)

                        _, preds = torch.max(outputs, 1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                            if use_tensorboard:
                                writer.add_scalar('Training Loss', loss, epoch)
                                writer.add_scalar('Training Accuracy', torch.sum(preds == labels.data).double() / len(labels), epoch)

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

                if phase == 'train':
                    train_acc_history.append(epoch_acc.item())
                    train_loss_history.append(epoch_loss)
                else:
                    #deep copying the model if its performance has improved
                    if epoch_acc > best_acc:
                        best_acc = epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())

                    #saving validation performance
                    val_acc_history.append(epoch_acc.item())
                    val_loss_history.append(epoch_loss)
                    #early stopping
                    if early_stopping:
                        if len(val_loss_history) > patience:
                            if (np.mean(val_loss_history[-patience:]) < stop_tol) or (np.all(np.array(val_loss_history[-8:]) > min(val_loss_history))):
                                print("Early stopping activated")
                                early_stop = True
                                final_epoch = epoch

                    if use_tensorboard:
                        writer.add_scalar('Validation Accuracy', epoch_acc, epoch)
                        writer.add_scalar('Validation Loss', epoch_loss, epoch)

            print("epoch {} complete".format(epoch))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    if use_tensorboard:
        #making sure all data is written to disk
        writer.flush()

    # load best model weights and return best model
    model.load_state_dict(best_model_wts)
    convergence_dict = {'train_acc': train_acc_history,
                        'train_loss': train_loss_history,
                        'val_acc': val_acc_history,
                        'val_loss': val_loss_history}

    if early_stopping:
        return model, convergence_dict, final_epoch
    else:
        return model, convergence_dict